In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './추출한_건물정보/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.csv')]
filenames

['enviro.csv',
 'epi.csv',
 'first_approved_date.csv',
 'floor_info_main_data_inter_has_primary_usage.csv',
 'floor_min_max_mean_median.csv',
 'new_structure_features.csv',
 'roof_fraction.csv',
 '총괄표제부상_epi점수.csv',
 '총괄표제부상_부속건축물면적.csv',
 '총괄표제부상_부속건축물수.csv',
 '총괄표제부상_연면적.csv',
 '총괄표제부상_외필지수.csv',
 '총괄표제부상_용적연면적.csv',
 '총괄표제부상_지능형점수.csv',
 '총괄표제부상_총기계식주차수.csv',
 '총괄표제부상_총세대수.csv',
 '총괄표제부상_총주차수.csv',
 '총괄표제부상_최초승인일.csv',
 '총괄표제부상_친환경점수.csv',
 '표제부상_부속건축물수.csv',
 '표제부상_연면적.csv',
 '표제부상_외필지수.csv',
 '표제부상_용적률산정연면적.csv',
 '표제부상_총주차수.csv']

In [3]:
wbdinfo_date_df = pd.read_csv(basedir + '총괄표제부상_최초승인일.csv')
print(wbdinfo_date_df.shape)
wbdinfo_date_df.head()

(44963, 3)


,지번주소,기준년월,사용승인년도
0,서울특별시 강남구 개포동 1193-5,201812,2018
1,서울특별시 강남구 개포동 1193-5,201912,2018
2,서울특별시 강남구 개포동 1193-5,202011,2018
3,서울특별시 강남구 개포동 12-0,201412,2012
4,서울특별시 강남구 개포동 12-0,201512,2012


In [4]:
wsub_df = pd.read_csv(basedir + '총괄표제부상_부속건축물수.csv')
print(wsub_df.shape)
wsub_df.head()

(44963, 3)


,지번주소,기준년월,부속건축물수
0,서울특별시 강남구 개포동 1193-5,201812,0
1,서울특별시 강남구 개포동 1193-5,201912,0
2,서울특별시 강남구 개포동 1193-5,202011,0
3,서울특별시 강남구 개포동 12-0,201412,3
4,서울특별시 강남구 개포동 12-0,201512,3


In [5]:
wmerge1_df = wbdinfo_date_df.merge(wsub_df, on=['지번주소', '기준년월'])
wmerge1_df.shape

(44963, 4)

In [6]:
bdinfo_date_df = pd.read_csv(basedir + 'first_approved_date.csv')
print(bdinfo_date_df.shape)
bdinfo_date_df.head()

(716540, 3)


,지번주소,기준년월,사용승인년도
0,서울특별시 마포구 서교동 484-16,201412,2004
1,서울특별시 동작구 상도동 302-28,201412,2008
2,서울특별시 도봉구 쌍문동 380-24,201412,2008
3,서울특별시 마포구 성산동 200-260,201412,2005
4,서울특별시 강동구 강일동 674-0,201412,2009


In [7]:
sub_df = pd.read_csv(basedir + '표제부상_부속건축물수.csv')
print(sub_df.shape)
sub_df.head()

(716540, 3)


,지번주소,기준년월,부속건축물_수
0,서울특별시 마포구 서교동 484-16,201412,0
1,서울특별시 동작구 상도동 302-28,201412,0
2,서울특별시 도봉구 쌍문동 380-24,201412,0
3,서울특별시 마포구 성산동 200-260,201412,0
4,서울특별시 강동구 강일동 674-0,201412,7


In [8]:
merge1_df = bdinfo_date_df.merge(sub_df, on=['지번주소', '기준년월'])
merge1_df.shape

(716540, 4)

In [9]:
wmerge1_df.head()

,지번주소,기준년월,사용승인년도,부속건축물수
0,서울특별시 강남구 개포동 1193-5,201812,2018,0
1,서울특별시 강남구 개포동 1193-5,201912,2018,0
2,서울특별시 강남구 개포동 1193-5,202011,2018,0
3,서울특별시 강남구 개포동 12-0,201412,2012,3
4,서울특별시 강남구 개포동 12-0,201512,2012,3


In [10]:
merge1_df.head()

,지번주소,기준년월,사용승인년도,부속건축물_수
0,서울특별시 마포구 서교동 484-16,201412,2004,0
1,서울특별시 동작구 상도동 302-28,201412,2008,0
2,서울특별시 도봉구 쌍문동 380-24,201412,2008,0
3,서울특별시 마포구 성산동 200-260,201412,2005,0
4,서울특별시 강동구 강일동 674-0,201412,2009,7


In [11]:
merge1_df.rename(columns={'부속건축물_수':'부속건축물수'}, inplace=True)

In [12]:
concat_df = pd.concat([merge1_df, wmerge1_df])
concat_df.shape

(761503, 4)

In [13]:
concat_df['사용승인년도'] = concat_df[['지번주소', '기준년월', '사용승인년도']].groupby(['지번주소', '기준년월']).transform('min')['사용승인년도']

In [14]:
concat_df['부속건축물수'] = concat_df[['지번주소', '기준년월', '부속건축물수']].groupby(['지번주소', '기준년월']).transform('max')['부속건축물수']

In [15]:
concat_df.drop_duplicates(inplace=True)
concat_df.shape

(717462, 4)

In [16]:
concat_df['기준년월'] = concat_df['기준년월'].apply(lambda x: int(str(x)[:4]))
concat_df.rename(columns={'기준년월':'기준년도'}, inplace=True)
concat_df.head()

,지번주소,기준년도,사용승인년도,부속건축물수
0,서울특별시 마포구 서교동 484-16,2014,2004,0
1,서울특별시 동작구 상도동 302-28,2014,2008,0
2,서울특별시 도봉구 쌍문동 380-24,2014,2008,0
3,서울특별시 마포구 성산동 200-260,2014,2005,0
4,서울특별시 강동구 강일동 674-0,2014,2009,7


In [17]:
def create_full_df_iteratively(df):
    # df is a pandas dataframe
    
    import pandas as pd
    import numpy as np
    import gc
    
    df = df.copy()
    
    dfs_list = []
    count = 0
    for addr in df['지번주소'].unique():
        addr_df = df[df['지번주소'] == addr]
        
        min_record_year = addr_df['기준년도'].min()
        max_record_year = addr_df['기준년도'].max()
        unique_record_years = addr_df['기준년도'].unique()

        unique_approv_years = sorted(addr_df['사용승인년도'].unique().tolist())
        min_approved_year = addr_df['사용승인년도'].min()
        max_approved_year = addr_df['사용승인년도'].max()

        if min_approved_year <= 2006:
            min_year = 2006
        else:
            min_year = min_approved_year

        years_for_concat = [y for y in range(min_year, max_record_year) if y not in unique_record_years]
        if years_for_concat == []:
            dfs_list.append(addr_df)
            continue

        rows_list = []
        for y in years_for_concat:
            empty_row = addr_df.iloc[[0],:]
            empty_row['기준년도'] = y
            empty_row.iloc[0, 2:] = np.nan
            rows_list.append(empty_row)

        empty_df = pd.concat(rows_list)

        addr_concat_df = pd.concat([addr_df, empty_df]).sort_values(['기준년도']).reset_index(drop=True)

        addr_concat_df.fillna(method='ffill', inplace=True)
        addr_concat_df.fillna(method='bfill', inplace=True)
        
        dfs_list.append(addr_concat_df)
        count += 1
        if count % 5000 == 0:
            gc.collect()
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    return concat_df

In [18]:
def parallelize(df, func, num_processors=6):
    # df is a pandas dataframe
    # func is the function to use
    # num_processors is the number of cpu cores
    
    data_split = []
    unique_addrs = df['지번주소'].unique()
    divided = int(len(unique_addrs) / num_processors)
    for i in range(num_processors):
        if i < (num_processors - 1):
            picked_addrs = unique_addrs[divided*i:divided*(i+1)]
        else:
            picked_addrs = unique_addrs[divided*i:]
        data_split.append(df[df['지번주소'].isin(picked_addrs)])
            
    pool = mp.Pool(num_processors)
    concat_df = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return concat_df

In [19]:
%%time
completed_df = parallelize(concat_df, create_full_df_iteratively)
print(completed_df.shape)
completed_df.head()

(1438766, 4)
Wall time: 4min 52s


,지번주소,기준년도,사용승인년도,부속건축물수
0,서울특별시 마포구 서교동 484-16,2006,2004.0,0.0
1,서울특별시 마포구 서교동 484-16,2007,2004.0,0.0
2,서울특별시 마포구 서교동 484-16,2008,2004.0,0.0
3,서울특별시 마포구 서교동 484-16,2009,2004.0,0.0
4,서울특별시 마포구 서교동 484-16,2010,2004.0,0.0


In [20]:
%%time
completed_df.drop(columns=['사용승인년도'], inplace=True)
completed_df.to_csv('./prepped_data/bdinfo_wbdinfo_subnum.csv', index=False)

Wall time: 2.61 s
